# Distinguishing Orthogonal Quantum States Workbook

This workbook describes the solutions to the problems offered in the "Distinguishing Orthogonal Quantum States" kata. Since the tasks are offered as programming problems, the explanations also cover some elements of Qiskit that might be non-obvious for a first-time user.

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

## Exercise 1. Distinguish |00⟩ and |11⟩

Both qubits in the input register are in the same state: for $\ket{00}$ each individual qubit is in state $\ket{0}$, for $\ket{11}$ each individual qubit is in state $\ket{1}$. Therefore, if we measure one qubit, we will know the state of the other qubit.

In other words, if the first qubit measures as `1`, we know that the qubits in the input register are in state $\ket{11}$, and if it measures as `0`, we know they are in state $\ket{00}$.

In [ ]:
def zerozero_or_oneone(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    circ.measure(qr[0], cr[0])

Alternatively, you can measure both qubits and return the decoder lambda that would convert measurement bit strings to state indices: "00" to $0$ and "11" to $1$.

In [ ]:
def zerozero_or_oneone(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    circ.measure(qr, cr)
    return lambda meas: 0 if meas == '00' else 1

## Exercise 2. Distinguish four basis states

Unlike in the previous task, this time measuring the first qubit won't give us any information on the second qubit, so we need to measure both qubits.

First, we measure both qubits in the input array and store the results in `cr[0]` and `cr[1]`. We can decode these results like this:

* `cr[0] = 0` and `cr[1] = 0`: we return $0 \cdot 2 + 0 = 0$
* `cr[0] = 1` and `cr[1] = 0`: we return $0 \cdot 2 + 1 = 1$
* `cr[0] = 0` and `cr[1] = 1`: we return $1 \cdot 2 + 0 = 2$
* `cr[0] = 1` and `cr[1] = 1`: we return $1 \cdot 2 + 1 = 1$

In other words, we treat the measurement results as the binary notation of the return value in little-endian notation, with the most significant bit stored in `cr[1]` and the least significant - in `cr[0]`.

In [ ]:
def measure_basis_state(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    circ.measure(qr, cr)

## Exercise 3. Distinguish two basis states given by bit strings

In this exercise, using the decoder can be very convenient! You can simply measure all bits and compare the bit string of measurement results with the given bit strings. Remember that Qiskit reverses the bit strings when representing measurement results; this means that you have to reverse the bit string of measurement results when comparing it with the given bit string.

In [ ]:
def measure_two_bitstrings(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister, bits0: list[bool], bits1: list[bool]) -> None:
    circ.measure(qr, cr)

    return lambda meas: 0 if [True if meas[ind] == '1' else False for ind in range(len(meas))][::-1] == bits0 else 1

## Exercise 4. Distinguish two superposition states given by lists of bit strings

Because all the bit strings are guaranteed to be different, and we are not limited in the number of measurements we can do, we can use a solution similar to the previous exercise.

When we measure all qubits of a certain superposition state, it collapses to one of the basis vectors that comprised the superposition. We can do exactly that and compare the resulting state to the given bit strings to see which array it belongs to. If it is in `bits0`, we were given the first state, otherwise it was the second state.

In [ ]:
def measure_four_bitstring_superpositions(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister, bits0: list[list[bool]], bits1: list[list[bool]]) -> None:
    circ.measure(qr, cr)

    return lambda meas: 0 if [True if meas[ind] == '1' else False for ind in range(len(meas))][::-1] in bits0 else 1

## Exercise 5. Distinguish two superposition states given by lists of bit strings in one measurement

To start with, we are looking for the index $Q$ where the two bit strings differ. It has two properties:

1. The value of all lists in `bits0` at the index $Q$ is either `True` or `False`, and the value of all lists in `bits1` at the index $Q$ is either `True` or `False` as well. If this is not the case, you cannot be sure that measuring the corresponding qubit will always return the same result.
   > For example, if you are given the state $\frac{1}{\sqrt2}\big(\ket{010} + \ket{011}\big)$, and if you measure the third qubit, $50\%$ of the time you will get $0$ and $50\%$ of the time $1$, therefore to get reliable information you want to measure one of the first two qubits.

2. This value is different for `bits0` and `bits1`.
   > Indeed, if you want to distinguish states $\frac{1}{\sqrt2}\big(\ket{010} + \ket{011}\big)$ and $\frac{1}{\sqrt2}\big(\ket{000} + \ket{001}\big)$, there are two qubits that will produce a fixed measurement result for each state - the first one and the second one. However, measuring the first qubit will give $0$ for both states, while measuring the second qubit will give $1$ for the first state and $0$ for the second state, allowing to distinguish the states.

To do this, we will iterate over all qubit indices, and for each of them we'll calculate the number of 1s in that position in `bits0` and `bits1`.

1. The first condition means that this count should equal 0 (if all bit strings have bit 0 in this position) or the length of the array of bit strings (if all bit strings have bit 1 in this position).
2. The second condition means that this count is different for `bits0` and `bits1`, i.e., one of the counts should equal 0 and another one - the length of the corresponding array of bit strings.

The last step is straightforward: using the index $Q$ we just found, we measure the qubit on that position. We need to keep in mind that there are two possible scenarios here:

1. `bits0[j][Q] = 0` and `bits1[j][Q] = 1`, in which case the measurement gives us the correct answer right away.
2. `bits0[j][Q] = 1` and `bits1[j][Q] = 0`, in which case the measurement gives us the opposite of the correct answer. To account for this, we need to either flip the state of the qubit before measuring it by applying the X gate to it, or to return a decoder that will make the correction.

In [ ]:
def measure_four_bitstring_superpositions_one(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister, bits0: list[list[bool]], bits1: list[list[bool]]) -> None:
    # Find Q
    for q in range(qr.size):
        count0 = sum([1 if bits[q] else 0 for bits in bits0])
        count1 = sum([1 if bits[q] else 0 for bits in bits1])
        if (count0 == len(bits0) and count1 == 0) or (count0 == 0 and count1 == len(bits1)):
            break

    # If the first state has 0 and the second has 1 in this position, we can measure right away.
    # Otherwise, we need to flip the bit before measurement (or return the decoder)
    if count1 == 0:
        circ.x(qr[q])
    circ.measure(qr[q], cr[0])

## Exercise 6. Distinguish |0...0⟩ state from W state

We can see that each basis state in the W state always has exactly one qubit in the $\ket{1}$ state, while in the $\ket{0...0}$ state, all qubits are in the $\ket{0}$ state.

We can use this to arrive to the solution: we will count the number of qubits that were measured in `One` state; if this number equals 1, we had a W state, if it equals 0, we know it was the $\ket{0...0}$ state.

In [ ]:
def zero_or_wstate(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    circ.measure(qr, cr)

    return lambda meas: 0 if sum([True if meas[ind] == '1' else False for ind in range(len(meas))]) == 0 else 1

## Exercise 7. Distinguish GHZ state from W state

As we've seen in the previous exercise, each of the basis states that form the W state will have exactly one qubit in the $\ket{1}$ state. Basis states that forms the GHZ state will either have all qubits in the $\ket{1}$ state or all qubits in the $\ket{0}$ state.

This means that if we count the number of qubits that were measured in the $\ket{1}$ state, we'll get 1 for the W state, and 0 or $N$ for the GHZ state. The code ends up almost the same as in the previous exercise (in fact, you can use this exact code to solve the previous exercise).

In [ ]:
def ghz_or_wstate(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    circ.measure(qr, cr)

    return lambda meas: 1 if sum([True if meas[ind] == '1' else False for ind in range(len(meas))]) == 1 else 0

## Exercise 8. Distinguish four Bell states

If the qubits are entangled in one of the Bell states, you can't simply measure individual qubits to distinguish the states: if you do, the first two states will both give you 00 or 11, and the last two: 01 or 10. We need to come up with a way to transform the original states to states that are easy to distinguish before measuring them.

First, let's take a look at the preparation of the Bell states starting with the $\ket{00}$ basis state.

> A more detailed discussion of preparing the Bell states can be found in the task "Prepare any Bell state" of the "Preparing Quantum States" kata.

The unitary transformation $\text{CNOT} \cdot (H \otimes I)$ (which corresponds to applying the $H$ gate to the first qubit, followed by applying the $\text{CNOT}$ gate with the first qubit as control and the second qubit as target) transforms the four basis vectors of the computational basis into the four Bell states.

To transform the Bell states back to the basis states, you can apply the *adjoint* of this transformation, which will undo its effects. In this case, both gates used are self-adjoint, so the adjoint transformation will require applying the same gates in reverse order (first $\text{CNOT}$, then $H$).

After this the original states will be transformed as follows:

| Return value | Original state | Maps to basis state |
| --- | --- | --- |
| 0 | $\ket{\Phi^{+}} = \frac{1}{\sqrt{2}} \big (\ket{00} + \ket{11}\big)$ | $\ket{00}$ |
| 1 | $\ket{\Phi^{-}} = \frac{1}{\sqrt{2}} \big (\ket{00} - \ket{11}\big)$ | $\ket{10}$ |
| 2 | $\ket{\Psi^{+}} = \frac{1}{\sqrt{2}} \big (\ket{01} + \ket{10}\big)$ | $\ket{01}$ |
| 3 | $\ket{\Psi^{-}} = \frac{1}{\sqrt{2}} \big (\ket{01} - \ket{10}\big)$ | $\ket{11}$ |

Measuring the qubits will produce exactly the return indices of the Bell states, in little-endian notation.

In [ ]:
def measure_bell_states(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    circ.cx(qr[0], qr[1])
    circ.h(qr[0])
    circ.measure(qr, cr)

## Exercise 9. Distinguish four two-qubit states

Similarly to the previous exercise, let's see whether these states can be converted back to the basis states.

To find a transformation that would convert the basis states to $\ket{S_0},...\ket{S_3}$, let's write out the coefficients of these states as column vectors side by side, so that they form a matrix.

$$\frac12 \begin{bmatrix} 1 & 1 & 1 & 1 \\ 1 & -1 & 1 & -1 \\ 1 & 1 & -1 & -1 \\ 1 & -1 & -1 & 1 \\ \ket{S_0} & \ket{S_1} & \ket{S_2} & \ket{S_3} \end{bmatrix}$$

Applying this matrix to each of the basis states will produce the given states. You can check explicitly that applying this transformation to the basis state $\ket{00}$ gives:

$$\frac{1}{2} \begin{bmatrix} 1 & 1 & 1 & 1 \\ 1 & -1 & 1 & -1 \\ 1 & 1 & -1 & -1 \\ 1 & -1 & -1 & 1 \end{bmatrix} \cdot \begin{bmatrix}1 \\ 0 \\ 0 \\ 0 \end{bmatrix} = 
\frac{1}{2}\begin{bmatrix}1 \\ 1 \\ 1 \\ 1 \end{bmatrix} = 
\frac{1}{2} \big(\ket{00} + \ket{01} + \ket{10} + \ket{11}\big) = \ket{S_0}$$

and similarly for the rest of the states.

Notice that the top left $2\times2$ block of this matrix is the same as the top right and the bottom left, and same as the bottom right block multiplied by $-1$. This means that we can represent this transformation as a tensor product of two $H$ gates:

$$ H \otimes H = 
\frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} \otimes \frac{1}{\sqrt{2}} \begin{bmatrix} 1 & 1 \\ 1 & -1 \end{bmatrix} = 
\frac{1}{2} \begin{bmatrix} 1 & 1 & 1 & 1 \\ 1 & -1 & 1 & -1 \\ 1 & 1 & -1 & -1 \\ 1 & -1 & -1 & 1 \end{bmatrix}  $$

Knowing how to prepare the given states, we can convert the input state back to the corresponding basis state using the adjoint of the preparation procedure, like we did in the previous exercise, and measure both qubits to get the answer.

In [ ]:
def measure_twoqubit_separable_states(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    circ.h(qr)
    circ.measure(qr, cr)

## Exercise 10. Distinguish four entangled two-qubit states

Here we can leverage the same method as used in the previous exercise: find a transformation that maps the basis states to the given states and apply its adjoint to the input state before measuring.

It is a lot harder to recognize the necessary transformation, though. The coefficient $\frac{1}{2}$ hints that there are still two $H$ gates involved, but this transformation is not a tensor product. After some experimentation you can find that the given states can be prepared by applying the $H$ gate to the first qubit of the matching Bell states:

| Basis state | Bell state | State after applying $H$ gate | Return value |
| --- | --- | --- | --- |
| $\ket{00}$ | $\ket{\Phi^{+}} = \frac{1}{\sqrt{2}} \big (\ket{00} + \ket{11}\big)$ | $\frac{1}{2} \big (\ket{00} + \ket{10} + \ket{01} - \ket{11}\big) = \ket{S_3}$ | $3$ |
| $\ket{10}$ | $\ket{\Phi^{-}} = \frac{1}{\sqrt{2}} \big (\ket{00} - \ket{11}\big)$ | $\frac{1}{2} \big (\ket{00} + \ket{10} - \ket{01} + \ket{11}\big) = \ket{S_2}$ | $2$ |
| $\ket{01}$ | $\ket{\Psi^{+}} = \frac{1}{\sqrt{2}} \big (\ket{01} + \ket{10}\big)$ | $\frac{1}{2} \big (\ket{00} - \ket{10} + \ket{01} + \ket{11}\big) = \ket{S_1}$ | $1$ |
| $\ket{11}$ | $\ket{\Psi^{-}} = \frac{1}{\sqrt{2}} \big (\ket{01} - \ket{10}\big)$ | $\frac{1}{2} \big (-\ket{00} + \ket{10} + \ket{01} + \ket{11}\big) = \ket{S_0}$  | $0$ |

In [ ]:
def measure_twoqubit_entangled_states(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    circ.h(qr[0])
    circ.cx(qr[0], qr[1])
    circ.h(qr[0])
    circ.measure(qr, cr)

    return lambda meas: 3 - int(meas, 2)

## Exercise 11. Distinguish two three-qubit states

Let's find a unitary transformation that converts the state $\ket{S_0}$ to the basis state $\ket{000}$. To do this, we first apply a unitary operation that maps the first state to the W state 
$\frac{1}{\sqrt{3}} \big(\ket{100} + \ket{010} + \ket{001}\big)$.

After this, we will use a convenient rotation gate $R_1$ (or $P$) which applies a relative phase to the $\ket{1}$ state and doesn't change the $\ket{0}$ state.
In matrix form: 

$$R_1(\theta) = \begin{bmatrix} 1 & 0 \\ 0 & e^{i\theta} \end{bmatrix} $$

We can convert $\ket{S_0}$ to $\ket{W}$ using a tensor product of gates
$I \otimes R_1(-\frac{2\pi}{3}) \otimes R_1(-\frac{4\pi}{3})$, where

* $I$ is the identity gate applied to qubit 0,
* $R_1(-\frac{2\pi}{3}) = \begin{bmatrix} 1 & 0 \\ 0 & \omega^{-1} \end{bmatrix}$, applied to qubit 1,
* $R_1(-\frac{4\pi}{3}) = \begin{bmatrix} 1 & 0 \\ 0 & \omega^{-2} \end{bmatrix}$, applied to qubit 2.

> Note that applying this operation to the state $\ket{S_1}$ converts it to $\frac{1}{\sqrt{3}} \big (\ket{100} + \omega \ket{010} + \omega^2 \ket{001} \big)$.

Now we can use adjoint of the state preparation routine for W state (from the exercise "Prepare W state" of the "Preparing Quantum States" kata), which will map the W state to the state $\ket{000}$ and the second state to some other state $\ket{S'_1}$.

> We don't need to do the math to figure out the exact state $\ket{S'_1}$ in which $\ket{S_1}$ will end up after those two transformations. Remember that our transformations are unitary, i.e., they preserve the inner products of vectors. Since the states $\ket{S_0}$ and $\ket{S_1}$ were orthogonal, their inner product $\braket{S_0|S_1} = 0$ is preserved when applying unitary transformations, and the states after the transformation will remain orthogonal.

The state $\ket{S'_1}$ is guaranteed to be orthogonal to the state $\ket{000}$, i.e., $\ket{S_1}$ gets mapped to a superposition that does not include basis state $\ket{000}$. To distinguish the states $\ket{000}$ and $\ket{S'_1}$, we measure all qubits; if all measurement results were 0, the state was $\ket{000}$ and we return 0, otherwise we return 1.

In [ ]:
def measure_threequbit_states(circ: QuantumCircuit, qr: QuantumRegister, cr: ClassicalRegister) -> None:
    from cmath import exp, pi
    from math import asin, sqrt
    from qiskit.circuit.library.standard_gates import RYGate    

    # Convert S0 to W
    angle = -2 * pi / 3
    circ.p(angle, qr[1])
    circ.p(2 * angle, qr[2])

    # Convert W to 000
    for ind in [2, 1]:
        circ.append(RYGate(-2 * asin(1 / sqrt(3 - ind))).control(ind, ctrl_state=0), qr[:ind + 1])    
    circ.ry(-2 * asin(1 / sqrt(3)), qr[0])

    # Measure
    circ.measure(qr, cr)

    return lambda meas: 0 if meas == "000" else 1